<a href="https://colab.research.google.com/github/mscholl96/mad-recime/blob/network_LSTM/network/LSTM/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

## Basic includes

In [1]:
!pip install ipython-autotime
%load_ext autotime

time: 2.54 ms (started: 2022-02-19 12:47:52 +00:00)


In [ ]:
!pip install word2vec

In [2]:
import word2vec
from collections import Counter # https://pymotw.com/2/collections/counter.html

import pandas as pd
import numpy as np
import itertools
import re
import os

import glob

time: 3.9 ms (started: 2022-02-19 12:47:52 +00:00)


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

dataPath = '/content/drive/MyDrive/TP2/Datasets/Recipe1M/'
import sys
sys.path.append(dataPath)

Mounted at /content/drive/
time: 17.9 s (started: 2022-02-19 12:47:52 +00:00)


## Import Data

In [4]:
TIMESTAMP = '2022_02_11'

time: 1.71 ms (started: 2022-02-19 12:48:10 +00:00)


In [5]:
baseFrame = pd.DataFrame()

smallSet = True

if(os.path.exists(dataPath + TIMESTAMP + '/recipes_valid_full.pkl')):
  baseFrame = pd.read_pickle(dataPath + TIMESTAMP + '/recipes_valid_full.pkl')
elif(smallSet == True):
  baseFrame = baseFrame.append(pd.read_pickle(glob.glob(dataPath + TIMESTAMP +  '/recipes_valid_*.pkl')[0]))
elif(len(glob.glob(dataPath + TIMESTAMP +  '/recipes_valid_*.pkl')) != 0):
  for file in glob.glob(dataPath + TIMESTAMP +  '/recipes_valid_*.pkl'):
    if not 'full' in file:
      baseFrame = baseFrame.append(pd.read_pickle(file))

baseFrame.head()

,title,ingredients,instructions
id,,,
000033e39b,Dilly Macaroni Salad Recipe,amount unit ingredient 0 1....,0 Cook macaroni according to package direct...
000035f7ed,Gazpacho,amount unit ingredient 0 8.0 ...,0 Add the tomatoes to a food processor with...
00003a70b1,Crunchy Onion Potato Bake,amount unit ingredient 0 2...,0 Preheat oven to 350 degrees Fah...
00004320bb,Cool 'n Easy Creamy Watermelon Pie,amount unit ingredient 0 3....,0 Dissolve Jello in boiling water. 1 ...
0000631d90,Easy Tropical Beef Skillet,amount unit ingredient 0...,"0 In a large skillet, toast the coconut ove..."


time: 38.3 s (started: 2022-02-19 12:48:10 +00:00)


In [6]:
def flatten_ing(row):
  row['amount'] = row['ingredients']['amount'].tolist()
  row['unit'] = row['ingredients']['unit'].tolist()
  row['ingredient'] = row['ingredients']['ingredient'].tolist()
  return row

baseFrame = baseFrame.apply(flatten_ing, axis=1)
baseFrame = baseFrame.drop(columns=['ingredients'])

baseFrame.head()

,title,instructions,amount,unit,ingredient
id,,,,,
000033e39b,Dilly Macaroni Salad Recipe,0 Cook macaroni according to package direct...,"[1.0, 1.0, 0.5, 0.5, 3.0, 0.5, 1.0, 0.75, 0.5]","[, , , , tablespoon, , tablespoon, teaspoon, t...","[elbow macaroni, american cheese, celery, gree..."
000035f7ed,Gazpacho,0 Add the tomatoes to a food processor with...,"[8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 3.0]","[, , , , , , , , ]","[tomatoes, kosher salt, red onion, green bell ..."
00003a70b1,Crunchy Onion Potato Bake,0 Preheat oven to 350 degrees Fah...,"[2.5, 1.5, 0.25, 1.0, 8.0, 1.0, 1.0]","[, , cup, , ounce, cup, cup]","[milk, water, butter, mashed potatoes, whole k..."
00004320bb,Cool 'n Easy Creamy Watermelon Pie,0 Dissolve Jello in boiling water. 1 ...,"[3.0, 0.25, 12.0, 2.0, 1.0]","[ounce, cup, ounce, cup, ]","[watermelon gelatin, boiling water, cool whip,..."
0000631d90,Easy Tropical Beef Skillet,"0 In a large skillet, toast the coconut ove...","[0.5, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 8.0, 16.0,...","[cup, pound-mass, tablespoon, , tablespoon, ta...","[shredded coconut, lean ground beef, fresh gar..."


time: 4min 3s (started: 2022-02-19 12:48:48 +00:00)


## Brainstorming
* not a simple sequence
* basically sequence of
  * ingredients
  * set of instructions --> instruction --> words
* maybe instead of instructions sequences, instructions could be flattened and split signs (.,!?;) also learned (in case learning sentence sequences is not possible)

Maybe stacked LSTMs are a solutions
* train how instructions in general are to be composed
* train how instructions are combined to a set of instructions

Input
* ingredients to be used as input
* to not train on instructions to appear according to ingredients sequence, maybe randomly shuffle ingredients but keep instruction set the same
* 

https://www.youtube.com/watch?v=A9QVYOBjZdY&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S&index=5&ab_channel=TensorFlow


## Differentiation
* Title generation based on ingredients
* instruction generation

## Import Torch

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.data import get_tokenizer # https://pytorch.org/text/stable/data_utils.html

torch.manual_seed(1)

# initialize tokenizer
tokenizer = get_tokenizer("basic_english")

time: 5.26 s (started: 2022-02-19 12:52:52 +00:00)


# Setup
https://closeheat.com/blog/pytorch-lstm-text-generation-tutorial

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0))

Tesla P100-PCIE-16GB
time: 3.31 ms (started: 2022-02-19 14:09:31 +00:00)


### Get Corpus

In [25]:
w2v_model = word2vec.load(dataPath + 'vocab.bin')
ingredientDict = {}
for voc in w2v_model.vocab:
     # Offset by 1 so empty fields can be 0
     ingredientDict.setdefault(voc, len(ingredientDict)+1)

ingredientDict['dilly'] # not to be used as dict misses words

KeyError: ignored

time: 4.64 s (started: 2022-02-19 13:14:28 +00:00)


In [12]:
def getCorpus(title, ingredient, instructions):
  titleTok = tokenizer(title)
  ingTok = tokenizer(','.join(ingredient))
  instTok = tokenizer('\n'.join(instructions))
  return np.array(ingTok + titleTok + instTok)

corpus = np.vectorize(getCorpus, otypes=[np.ndarray])(baseFrame['title'], baseFrame['ingredient'], baseFrame['instructions'])
corpus = np.concatenate(corpus)

time: 14.1 s (started: 2022-02-19 12:53:14 +00:00)


In [24]:
unique = sorted(Counter(corpus), key=Counter(corpus).get, reverse=True)

index_to_word = {index: word for index, word in enumerate(unique)}
word_to_index = {word: index for index, word in enumerate(unique)}

time: 21.8 s (started: 2022-02-19 13:10:36 +00:00)


###  Sequences

In [14]:
def getTitleSequence(title, ingredient):
  titleTok = tokenizer(title)
  ingTok = tokenizer(','.join(ingredient))
  return np.array(ingTok + titleTok)

titleSeq = np.vectorize(getTitleSequence, otypes=[np.ndarray])(baseFrame['title'], baseFrame['ingredient'])
titleSeq[0]

array(['elbow', 'macaroni', ',', 'american', 'cheese', ',', 'celery', ',',
       'green', 'peppers', ',', 'pimentos', ',', 'mayonnaise', ',',
       'vinegar', ',', 'salt', ',', 'dry', 'dill', 'weed', 'dilly',
       'macaroni', 'salad', 'recipe'], dtype='<U10')

time: 4.09 s (started: 2022-02-19 12:53:33 +00:00)


In [15]:
def getInstSequence(ingredient, instructions):
  ingTok = tokenizer(','.join(ingredient))
  instTok = tokenizer('\n'.join(instructions))
  return np.array(ingTok + instTok)

instSeq = np.vectorize(getInstSequence, otypes=[np.ndarray])(baseFrame['ingredient'], baseFrame['instructions'])
instSeq[0]

array(['elbow', 'macaroni', ',', 'american', 'cheese', ',', 'celery', ',',
       'green', 'peppers', ',', 'pimentos', ',', 'mayonnaise', ',',
       'vinegar', ',', 'salt', ',', 'dry', 'dill', 'weed', 'cook',
       'macaroni', 'according', 'to', 'package', 'directions', 'drain',
       'well', '.', 'cold', '.', 'combine', 'macaroni', ',', 'cheese',
       'cubes', ',', 'celery', ',', 'green', 'pepper', 'and', 'pimento',
       '.', 'blend', 'together', 'mayonnaise', 'or', 'possibly', 'salad',
       'dressing', ',', 'vinegar', ',', 'salt', 'and', 'dill', 'weed',
       'add', 'in', 'to', 'macaroni', 'mix', '.', 'toss', 'lightly', '.',
       'cover', 'and', 'refrigeratewell', '.', 'serve', 'salad', 'in',
       'lettuce', 'lined', 'bowl', 'if', 'you', 'like', '.', 'makes', '6',
       'servings', '.'], dtype='<U15')

time: 11.4 s (started: 2022-02-19 12:53:37 +00:00)


### Dataset

In [ ]:
# Data (https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class)
class TitleDataset(torch.utils.data.Dataset):
  """
  Create Dataset class for title creation training
  """
    def __init__(self, hyperparams, data):
      self.hyperparams = hyperparams

      # dataset split into word sequences required for training
      self.wordSeq = np.vectorize(getTitleSequence, otypes=[np.ndarray])(baseFrame['title'], baseFrame['ingredient'])

      # list of all words in dataset
      self.words = np.concatenate(np.vectorize(getCorpus, otypes=[np.ndarray])(baseFrame['title'], baseFrame['ingredient'], baseFrame['instructions']))

      # list of all words in dataset
      self.unique = sorted(Counter(corpus), key=Counter(corpus).get, reverse=True)

      # https://closeheat.com/blog/pytorch-lstm-text-generation-tutorial
      self.index_to_word = {index: word for index, word in enumerate(unique)}
      self.word_to_index = {word: index for index, word in enumerate(unique)}

    def getCorpus(title, ingredient, instructions):
      titleTok = tokenizer(title)
      ingTok = tokenizer(','.join(ingredient))
      instTok = tokenizer('\n'.join(instructions))
      return np.array(ingTok + titleTok + instTok)

    def getTitleSequence(title, ingredient):
      titleTok = tokenizer(title)
      ingTok = tokenizer(','.join(ingredient))
      return np.array(ingTok + titleTok)

    def __len__(self):
        return len(self.words_indexes) - self.args.sequence_length

    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.args.sequence_length]),
            torch.tensor(self.words_indexes[index+1:index+self.args.sequence_length+1]),
        )



In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(
        self
    ):
        self.words = self.load_words()
        self.uniq_words = self.get_uniq_words()

        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        self.words_indexes = [self.word_to_index[w] for w in self.words]

    def load_words(self):
        train_df = pd.read_csv('https://raw.githubusercontent.com/amoudgl/short-jokes-dataset/master/data/reddit-cleanjokes.csv')
        text = train_df['Joke'].str.cat(sep=' ')
        return text.split(' ')

    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.words_indexes) - 4

    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+4]),
            torch.tensor(self.words_indexes[index+1:index+4+1]),
        )

In [ ]:
class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 128
        self.embedding_dim = 128
        self.num_layers = 3

        n_vocab = len(dataset.uniq_words)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))

In [ ]:
import argparse
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader

def train(dataset, model, args):
    model.train()

    dataloader = DataLoader(dataset, batch_size=args.batch_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(args.max_epochs):
        state_h, state_c = model.init_state(args.sequence_length)

        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()

            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()

            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })

In [ ]:
def predict(dataset, model, text, next_words=100):
    model.eval()

    words = text.split(' ')
    state_h, state_c = model.init_state(len(words))

    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.index_to_word[word_index])

    return words

In [ ]:
dataset = Dataset()
# model = Model(dataset)
# train(dataset, model, args)
# print(predict(dataset, model, text='Knock knock. Whos there?'))
dataset.__getitem__(4)